<a href="https://colab.research.google.com/github/alishraf12/recommender-system/blob/master/MLtaskv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
ds = pd.read_csv("jobs_data.csv") #you can plug in your own list of products or movies or books here as csv file#
dv = pd.read_csv("jobs_data.csv")
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')


In [0]:
!pip install scikit-surprise
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD

import warnings; warnings.simplefilter('ignore')

     |████████████████████████████████| 6.5MB 2.5MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1678237 sha256=2ebe0e002717ebefcd5e206537cfc4cc82b3be36afb6679b5ccca25c48d2420b
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [0]:
tfidf_matrix = tf.fit_transform(ds['title'])
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
results = {} # dictionary created to store the result in a dictionary format (ID : (Score,item_id))#
for idx, row in ds.iterrows():

  similar_indices = cosine_similarities[idx].argsort()[:-5:-1] 
  similar_items = [(cosine_similarities[idx][i], ds['title'][i]) for i in similar_indices]
  results[row['title']] = similar_items[1:] #iterates through all the rows

In [0]:
def item(title):
    return ds.loc[ds['title'] == title]['industry'].tolist()[0]
def recommend(title, num):
    if (num == 0):
        print("Unable to recommend any book as you have not chosen the number of book to be recommended")
    elif (num==1):
        print("Recommending " + str(num) + " book similar to " + item(title))
        
    else :
        print("Recommending " + str(num) + " books similar to " + item(title))
        
    print("----------------------------------------------------------")
    recs = results[title][:num]
    for rec in recs:      
      print("You may also like to read: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")

#the first argument in the below function to be passed is the id of the book, second argument is the number of books you want to be recommended#
recommend('PHP Full-Stack - Joomla Expert',2)

Recommending 2 books similar to ['Information Technology Services']
----------------------------------------------------------
You may also like to read: ['Information Technology Services'] (score:0.9999999999999999)
You may also like to read: ['Information Technology Services'] (score:0.9999999999999999)


# Data Cleaning

In [0]:
def clean_2(data):
  pos=[]
  count = 0
  str_count=0
  for i in data['title']:
    str_count = str_count + 1
    if '-' in i:
      count = count + 1
      if count == 2:
       post=(i.index('-'))
       dect={'str_count':str_count,' index':post}
       count=0
    
  for key,value in dect.items():
    print(key)
    print(value)



In [0]:
clean_2(ds)

str_count
10869
 index
39


In [0]:
def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

In [0]:
def clean_second_dash(data):
  x=[]
  str_count=0
  dect={'str_count':str_count,' index':x}
  for i in data['title']:
    position = find_nth(i,'-',2)
    string=dv['title'][str_count]
    if position > 0:
     data['title'][str_count] = string.replace(string[position:],"")
    str_count = str_count + 1
  
  


In [0]:
clean_second_dash(dv)


In [0]:

dv['title'][62]

'Administrative Assistant - Intern '

In [0]:
def display_all(data):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(data)  

In [0]:
display_all(ds)

,Unnamed: 0,title,jobFunction,industry
0,0,Full Stack PHP Developer,"['Engineering - Telecom/Technology', 'IT/Softw...","['Computer Software', 'Marketing and Advertisi..."
1,1,CISCO Collaboration Specialist Engineer,"['Installation/Maintenance/Repair', 'IT/Softwa...",['Information Technology Services']
2,2,Senior Back End-PHP Developer,"['Engineering - Telecom/Technology', 'IT/Softw...","['Computer Software', 'Computer Networking']"
3,3,UX Designer,"['Creative/Design/Art', 'IT/Software Developme...","['Computer Software', 'Information Technology ..."
4,4,Java Technical Lead,"['Engineering - Telecom/Technology', 'IT/Softw...","['Computer Software', 'Information Technology ..."
...,...,...,...,...
10865,10865,Odoo Developer,"['IT/Software Development', 'Engineering - Tel...","['Real Estate/Property Management', 'Manufactu..."
10866,10866,Senior PHP Developer - Alexandria,"['Engineering - Telecom/Technology', 'IT/Softw...",['Information Technology Services']
10867,10867,Sales & Account Manager (Real Estate / Medical...,"['Business Development', 'Sales/Retail', 'Cust...","['Real Estate/Property Management', 'Retail', ..."
10868,10868,Technical and Network Support Engineer - Alexa...,"['Installation/Maintenance/Repair', 'Engineeri...","['Computer Software', 'Education', 'Healthcare..."


In [0]:
def clean_first_dash(data):
  x=[]
  str_count=0
  dect={'str_count':str_count,' index':x}
  for i in data['title']:
    position = find_nth(i,' - ',1)
    string=dv['title'][str_count]
    if position > 0:
     data['title'][str_count] = string.replace(string[position:],"")
    str_count = str_count + 1
  
clean_first_dash(dv)


In [0]:
#dv['title'][8]



# Algorithm IMP

In [0]:
dv.head()

,Unnamed: 0,title,jobFunction,industry
0,0,Full Stack PHP Developer,"['Engineering - Telecom/Technology', 'IT/Softw...","['Computer Software', 'Marketing and Advertisi..."
1,1,CISCO Collaboration Specialist Engineer,"['Installation/Maintenance/Repair', 'IT/Softwa...",['Information Technology Services']
2,2,Senior Back End-PHP Developer,"['Engineering - Telecom/Technology', 'IT/Softw...","['Computer Software', 'Computer Networking']"
3,3,UX Designer,"['Creative/Design/Art', 'IT/Software Developme...","['Computer Software', 'Information Technology ..."
4,4,Java Technical Lead,"['Engineering - Telecom/Technology', 'IT/Softw...","['Computer Software', 'Information Technology ..."


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')

In [0]:
tfidf_matrix = tfidf.fit_transform(dv['jobFunction'])

In [0]:
tfidf_matrix.shape

(10870, 78)

In [0]:
from sklearn.metrics.pairwise import linear_kernel


In [0]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)



In [0]:
indices = pd.Series(dv.index, index=dv['title']).drop_duplicates()


In [0]:
def get_recommendations(title, cosine_sim=cosine_sim):
   
    idx = indices[title]                           
    
    sim_scores = list(enumerate(cosine_sim[idx]))       
    
   
    sim_scores = sorted(sim_scores, key=lambda x: x[0])    
    #sim_scores = sorted(sim_scores,reverse=True)
    sim_scores = sim_scores[1:5] 
    #print(sim_scores)

    blog_indices = [i[0] for i in sim_scores]               

    
    return dv['jobFunction'].iloc[blog_indices]              

In [0]:
x = get_recommendations("Full Stack PHP Developer")
for i in reversed(range(1,5)):
  print(x[i])

['Engineering - Telecom/Technology', 'IT/Software Development']
['Creative/Design/Art', 'IT/Software Development']
['Engineering - Telecom/Technology', 'IT/Software Development']
['Installation/Maintenance/Repair', 'IT/Software Development', 'Engineering - Telecom/Technology']


In [0]:
input_="php full stack"
#tfidf_matrix_input = tfidf.fit_transform(input_)
tfidf_matrix_title=tfidf.fit_transform(dv['title'])
cosine_sim_input=linear_kernel(tfidf_matrix_title,tfidf_matrix_title)

In [0]:

def get_recommendations_title(title, cosine_sim=cosine_sim):
   
    idx = indices[title]                           
    
    sim_scores = list(enumerate(cosine_sim[idx]))       
    
   
    sim_scores = sorted(sim_scores, key=lambda x: x[0])    
    #sim_scores = sorted(sim_scores,reverse=True)
    sim_scores = sim_scores[1:5] 
    #print(sim_scores)

    blog_indices = [i[0] for i in sim_scores]               

    
    return dv['title'].iloc[blog_indices]           

In [0]:
z = ["Full Stack PHP Developer"]
get_recommendations_title(z)

1    CISCO Collaboration Specialist Engineer
2              Senior Back End-PHP Developer
3                                UX Designer
4                        Java Technical Lead
Name: title, dtype: object

In [0]:
tfidf_matrix_inp=tfidf.transform(z)

In [0]:
z


['Full Stack PHP Developer']